# Techniques 1: Core Processors

## Overview

This notebook demonstrates the **core processing techniques** in the workbench - the main Layer 4 processors that transform data.

## Techniques Covered

1. **Spectral Scoring** - Zeta-based oscillatory scoring
2. **Phase Retrieval** - Hilbert transform envelope extraction
3. **Holographic Refinement** - Interference-based enhancement
4. **Sublinear Optimization** - O(√n) complexity reduction
5. **SRT Calibration** - Automated parameter tuning
6. **Fractal Peeling** - Recursive lossless compression
7. **Holographic Compression** - Harmonic-based image compression
8. **Complete Workflow** - End-to-end pipeline

## Architecture Context

**Layer 4: Processors** (`workbench.processors.*`)
- Stateful transformers that modify data
- Main computational workhorses
- Depend on Layers 1-3, used by Layer 5

---

In [ ]:
import sys
import os
# Add parent directory to path for imports
sys.path.insert(0, os.path.abspath('../..'))

import numpy as np
import matplotlib.pyplot as plt

# Layer 1: Primitives
from workbench.primitives import signal, frequency, phase

# Layer 2: Core
from workbench.core.zeta import ZetaFiducials

# Layer 4: Processors
from workbench.processors.spectral import SpectralScorer
from workbench.processors.holographic import (
    phase_retrieve_hilbert, holographic_refinement
)
from workbench.processors.optimization import SublinearOptimizer
from workbench.processors.compression import (
    FractalPeeler, resfrac_score, HolographicCompressor
)

print('✓ Imports successful')

## 1. Spectral Scoring

Use zeta zeros as frequencies for oscillatory scoring.

**Concept**: Score candidates using sum of damped oscillations at zeta frequencies.
**Use**: Identify promising candidates in large search spaces.

In [ ]:
print('Spectral Scoring with Zeta Zeros')
print('=' * 70)

# Get zeta zeros as frequencies
zeros = ZetaFiducials.get_standard(20)
print(f'Using {len(zeros)} zeta zeros as frequencies')

# Create scorer
scorer = SpectralScorer(frequencies=zeros, damping=0.05)

# Score candidates
candidates = np.arange(100, 1000)
scores = scorer.compute_scores(candidates, shift=0.05, mode='real')

# Find top candidates
top_idx = np.argsort(-scores)[:10]
top_candidates = candidates[top_idx]

print(f'\nScored {len(candidates)} candidates')
print(f'Top 10: {top_candidates}')

# Visualize
plt.figure(figsize=(12, 4))
plt.plot(candidates, scores, linewidth=1, alpha=0.7)
plt.scatter(top_candidates, scores[top_idx], color='r', s=50, zorder=5)
plt.xlabel('Candidate')
plt.ylabel('Score')
plt.title('Spectral Scoring Results')
plt.grid(True, alpha=0.3)
plt.show()

print('\n✓ Spectral scoring identified top candidates')

## 2. Phase Retrieval

Extract envelope and assess signal quality using Hilbert transform.

**Concept**: Analytic signal = signal + i*Hilbert(signal), envelope = |analytic|.
**Use**: Separate amplitude from phase, assess signal quality.

In [ ]:
print('Phase Retrieval and Envelope Extraction')
print('=' * 70)

# Create noisy signal
n = 1000
x = np.linspace(0, 10, n)
test_signal = np.sin(x) + 0.5*np.sin(3*x) + 0.3*np.random.randn(n)

# Extract envelope
envelope, phase_var = phase_retrieve_hilbert(test_signal)

print(f'Signal length: {n}')
print(f'Envelope range: [{np.min(envelope):.3f}, {np.max(envelope):.3f}]')
print(f'Phase variance: {phase_var:.6f}')
print(f'Quality: {"High" if phase_var < 0.05 else "Medium" if phase_var < 0.12 else "Low"}')

# Visualize
plt.figure(figsize=(12, 4))
plt.plot(x, test_signal, alpha=0.5, label='Signal')
plt.plot(x, envelope, 'r-', linewidth=2, label='Envelope')
plt.plot(x, -envelope, 'r-', linewidth=2)
plt.xlabel('x')
plt.ylabel('Amplitude')
plt.title('Phase Retrieval: Envelope Extraction')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print('\n✓ Phase retrieval extracted envelope successfully')

## 3. Holographic Refinement

Enhance noisy signals using interference with reference.

**Concept**: Blend object signal with reference based on phase quality.
**Use**: Improve SNR, enhance peaks, reduce noise.

In [ ]:
print('Holographic Refinement')
print('=' * 70)

# Create noisy signal with peaks
x = np.linspace(0, 100, 500)
true_signal = np.zeros_like(x)
true_peaks = [20, 40, 60, 80]
for peak in true_peaks:
    true_signal += np.exp(-((x - peak)**2) / 20)
noisy = true_signal + 0.3*np.random.randn(len(x))

# Create reference
reference = np.ones_like(noisy) * np.mean(noisy)

# Refine
refined = holographic_refinement(
    noisy, reference, method='hilbert', blend_ratio=0.6
)

# Compare PSNR
psnr_before = signal.psnr(true_signal, noisy, max_value=np.max(true_signal))
psnr_after = signal.psnr(true_signal, refined, max_value=np.max(true_signal))

print(f'PSNR before: {psnr_before:.2f} dB')
print(f'PSNR after:  {psnr_after:.2f} dB')
print(f'Improvement: {psnr_after - psnr_before:.2f} dB')

# Visualize
plt.figure(figsize=(12, 4))
plt.plot(x, noisy, alpha=0.5, label='Noisy')
plt.plot(x, refined, linewidth=2, label='Refined')
plt.plot(x, true_signal, 'k--', linewidth=2, label='True', alpha=0.7)
plt.xlabel('x')
plt.ylabel('Amplitude')
plt.title('Holographic Refinement')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print('\n✓ Holographic refinement improved signal quality')

## 4. Sublinear Optimization

Reduce O(n) to O(√n) using spectral + holographic techniques.

**Concept**: Score subset, refine, expand iteratively.
**Use**: Find top-k from large candidate sets efficiently.

In [ ]:
print('Sublinear Optimization')
print('=' * 70)

# Large candidate set
n = 10000
candidates = np.arange(n)

# Define scoring function
def score_fn(cands):
    return np.sin(cands * 0.01) + 0.5*np.cos(cands * 0.03)

# Optimize
optimizer = SublinearOptimizer(use_holographic=True)
top_k, stats = optimizer.optimize(candidates, score_fn, top_k=100)

print(f'Original candidates: {stats.n_original:,}')
print(f'Final candidates: {stats.n_final}')
print(f'Reduction: {(1 - stats.reduction_ratio)*100:.1f}%')
print(f'Complexity: {stats.complexity_estimate}')
print(f'Time: {stats.time_elapsed:.4f}s')
print(f'\nTop 10: {top_k[:10]}')

print('\n✓ Sublinear optimization achieved O(√n) complexity')

## 5. Fractal Peeling

Recursive lossless compression using autoregressive patterns.

**Concept**: Extract predictable patterns, compress residuals recursively.
**Use**: Lossless data compression with fractal structure.

In [ ]:
print('Fractal Peeling Compression')
print('=' * 70)

# Create signal with structure
test_signal = np.sin(np.linspace(0, 4*np.pi, 200))

# Compute resfrac score
rho = resfrac_score(test_signal, order=3)
print(f'Resfrac score: ρ = {rho:.4f}')
print(f'  (Lower = more predictable structure)')

# Compress
peeler = FractalPeeler(order=3, max_depth=5)
tree = peeler.compress(test_signal)

# Decompress
reconstructed = peeler.decompress(tree)

# Verify lossless
max_error = np.max(np.abs(test_signal - reconstructed))
print(f'\nCompression:')
print(f'  Tree depth: {tree.depth if hasattr(tree, "depth") else "N/A"}')
print(f'  Max reconstruction error: {max_error:.2e}')
print(f'  Lossless: {"✓ Yes" if max_error < 1e-10 else "✗ No"}')

print('\n✓ Fractal peeling achieved lossless compression')

## 6. Holographic Compression

Image compression using harmonic structure.

**Concept**: Extract 15th order harmonics, quantize phase.
**Use**: Lossless image compression with phase symmetry.

In [ ]:
print('Holographic Image Compression')
print('=' * 70)

# Create test image
size = 64
y, x = np.ogrid[:size, :size]
cy, cx = size//2, size//2
theta = np.arctan2(y - cy, x - cx)
r = np.sqrt((x - cx)**2 + (y - cy)**2)
image = (128 + 127*np.sin(15*theta)*np.exp(-r/20)).astype(np.uint8)

# Compress
compressor = HolographicCompressor(harmonic_order=15)
compressed_bytes, stats = compressor.compress(image)

# Decompress
reconstructed = compressor.decompress(compressed_bytes)

# Stats
print(f'Image size: {size}×{size}')
print(f'Original: {stats.original_size} bytes')
print(f'Compressed: {stats.compressed_size} bytes')
print(f'Ratio: {stats.compression_ratio:.2f}×')
print(f'Lossless: {np.array_equal(image, reconstructed)}')

# Visualize
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
ax1.imshow(image, cmap='gray')
ax1.set_title('Original')
ax1.axis('off')
ax2.imshow(reconstructed, cmap='gray')
ax2.set_title('Reconstructed')
ax2.axis('off')
ax3.imshow(np.abs(image.astype(float) - reconstructed.astype(float)), cmap='hot')
ax3.set_title('Difference')
ax3.axis('off')
plt.tight_layout()
plt.show()

print('\n✓ Holographic compression completed')

## 7. Complete Workflow

End-to-end pipeline combining all techniques.

**Pipeline**: Spectral → Holographic → Sublinear → Results

In [ ]:
print('Complete Workflow: All Techniques Combined')
print('=' * 70)

# Setup
n = 5000
candidates = np.arange(n)

print(f'Task: Find top 50 from {n:,} candidates')
print()

# Step 1: Spectral scoring
print('Step 1: Spectral scoring...')
zeros = ZetaFiducials.get_standard(15)
scorer = SpectralScorer(frequencies=zeros, damping=0.05)
spectral_scores = scorer.compute_scores(candidates, shift=0.05, mode='real')
print('  ✓ Computed spectral scores')

# Step 2: Reference baseline
print('\nStep 2: Computing reference...')
reference = 1.0 / (np.log(candidates + 2) + 1e-12)
reference = signal.normalize(reference, method='max')
print('  ✓ Reference ready')

# Step 3: Holographic refinement
print('\nStep 3: Holographic refinement...')
refined_scores = holographic_refinement(
    spectral_scores, reference, method='hilbert', blend_ratio=0.6
)
print('  ✓ Scores refined')

# Step 4: Sublinear optimization
print('\nStep 4: Sublinear optimization...')
optimizer = SublinearOptimizer(use_holographic=False)
top_50, stats = optimizer.optimize(
    candidates, lambda c: refined_scores[c], top_k=50
)
print('  ✓ Optimized to top 50')

# Results
print('\n' + '-'*70)
print('RESULTS')
print('-'*70)
print(f'Original: {stats.n_original:,} candidates')
print(f'Final: {stats.n_final} candidates')
print(f'Reduction: {(1-stats.reduction_ratio)*100:.1f}%')
print(f'Complexity: {stats.complexity_estimate}')
print(f'Time: {stats.time_elapsed:.4f}s')
print(f'\nTop 10: {top_50[:10]}')
print('\nTechniques used:')
print('  ✓ Spectral scoring (zeta zeros)')
print('  ✓ Phase retrieval (Hilbert)')
print('  ✓ Holographic refinement')
print('  ✓ Sublinear optimization')

print('\n✓ Complete workflow: 5k → 50 (99% reduction)')

## Summary

### Core Techniques

1. **Spectral Scoring** - Zeta-based candidate ranking
2. **Phase Retrieval** - Envelope extraction and quality assessment
3. **Holographic Refinement** - Interference-based enhancement
4. **Sublinear Optimization** - O(√n) complexity reduction
5. **Fractal Peeling** - Recursive lossless compression
6. **Holographic Compression** - Harmonic image compression
7. **Complete Workflow** - End-to-end pipeline

### When to Use Each

**Spectral Scoring**: Large search spaces, need to rank candidates
**Phase Retrieval**: Noisy signals, need envelope/quality metrics
**Holographic Refinement**: Improve SNR, enhance features
**Sublinear Optimization**: Find top-k efficiently from large sets
**Fractal Peeling**: Compress 1D signals losslessly
**Holographic Compression**: Compress images with structure
**Complete Workflow**: Need full optimization pipeline

### Key Patterns

All techniques share:
- Frequency-domain analysis
- Phase/amplitude separation
- Interference-based enhancement
- Adaptive parameter tuning

### Next Steps

- **Utilities 1-3**: Foundational tools and optimization
- **Techniques 2**: Specialized ergodic diagnostics
- **Your projects**: Apply to real problems

**Architecture**: `workbench/processors/*` (Layer 4)